In [2]:
pip install langchain langchain_community faiss-gpu langgraph langchain-openai

In [3]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import chat_agent_executor
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

search = TavilySearchResults(max_results=2)

# 加载网页内容
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
# 分割文本
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
# 生成嵌入向量并存储在FAISS向量数据库中
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
"langsmith_search",
"Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

tools = [search, retriever_tool]

model = ChatOpenAI(model="gpt-3.5-turbo")


In [6]:
prompt = ChatPromptTemplate.from_messages(
[
  ("user", "{text}")
]
)
parser = StrOutputParser()
model_with_tool=model.bind_tools(tools)
chain = prompt | model_with_tool | parser
chain.invoke({"text":"hi"})

'Hello! How can I assist you today?'

In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)
config = {"configurable": {"thread_id": "abc1234"}}
response = agent_executor.invoke({"messages": [HumanMessage(content="how can langsmith help with testing?")]},config)
response["messages"]

[HumanMessage(content='how can langsmith help with testing?', id='42f8ece5-2565-41bc-8cb5-e7a509bc51a7'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WNwDKIGRmfouNFHzbngoW0YO', 'function': {'arguments': '{"query":"how can LangSmith help with testing"}', 'name': 'langsmith_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 135, 'total_tokens': 156}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-eadc3621-84fb-493e-bd07-2bc66b6337ee-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'how can LangSmith help with testing'}, 'id': 'call_WNwDKIGRmfouNFHzbngoW0YO'}]),
 ToolMessage(content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangS

In [8]:
response = agent_executor.invoke({"messages": [HumanMessage(content="what was my last question?")]},config)
response["messages"]

[HumanMessage(content='how can langsmith help with testing?', id='42f8ece5-2565-41bc-8cb5-e7a509bc51a7'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"how can LangSmith help with testing"}', 'name': 'langsmith_search'}, 'id': 'call_WNwDKIGRmfouNFHzbngoW0YO', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 135, 'total_tokens': 156}}, id='run-eadc3621-84fb-493e-bd07-2bc66b6337ee-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'how can LangSmith help with testing'}, 'id': 'call_WNwDKIGRmfouNFHzbngoW0YO'}]),
 ToolMessage(content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangS

In [9]:
from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict, List
from langchain_core.outputs import LLMResult
from langchain_core.messages import BaseMessage
class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        print(f"Chain {serialized.get('name')} started")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")
callbacks = [LoggingHandler()]
config = {"configurable": {"thread_id": "abc1234"},"callbacks": callbacks}
response = agent_executor.invoke({"messages": [HumanMessage(content="what was my last question?")]}, config=config)

Chain LangGraph started
Chain None started
Chain ended, outputs: {'messages': [HumanMessage(content='what was my last question?')]}
Chain RunnableSequence started
Chain None started
Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='Your last question was "what was my last question?"', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Your last question was "what was my last question?"', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 1100, 'total_tokens': 1112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5443d6a7-1fdb-4f61-836b-15859591a4c8-0'))]] llm_output={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 1100, 'total_tokens': 1112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None} run=None
Chain ended, outputs: {'messages': [AIMessage(content='Your last question was "what was my last question?"', res